In [1]:
# load data
from data.data_collection import load_credit_data
df =  load_credit_data()
original_df = df.copy()
df_eda = df.copy()

In [2]:
from data.data_preparation import *
# split columns into categorical and continuous
splitted_columns = split_columns(df)
categorical_columns = splitted_columns['categorical_columns']
continuous_columns = splitted_columns['continuous_columns']
print(f"Categorical Columns\n{categorical_columns}")
print('='*300)
print(f"Continuous Columns\n{continuous_columns}")

Categorical Columns
['marital_status', 'april_delay', 'july_delay', 'may_delay', 'august_delay', 'sept_delay', 'education', 'Y', 'gender', 'june_delay']
Continuous Columns
['credit_amount', 'sept_bill', 'august_bill', 'july_bill', 'june_bill', 'may_bill', 'april_bill', 'sept_payment', 'august_payment', 'july_payment', 'june_payment', 'may_payment', 'april_payment', 'age']


In [3]:
# data cleaning
df = remove_nulls(df)
df = remove_duplicates(df)

In [4]:
df

,credit_amount,gender,education,marital_status,age,sept_delay,august_delay,july_delay,june_delay,may_delay,...,june_bill,may_bill,april_bill,sept_payment,august_payment,july_payment,june_payment,may_payment,april_payment,Y
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [5]:
from model_selection.model_training import get_train_test_data


X_train,X_test,y_train,y_test=get_train_test_data(X=df.drop(columns="Y"), y=df.Y, stratify=True)

In [6]:
# We will split the data before feat eng

# then feature engineering
from feature_preparation.feature_engineering import get_scaler, scale_data,create_total_delays, \
create_payment_consistency,transform_credit_amount 
X_train_with_featured = create_payment_consistency(X_train)
X_train_with_featured = create_total_delays(X_train)
X_train_with_featured = transform_credit_amount(X_train)
print(X_train_with_featured.head())

# now let us scale
scaler = get_scaler(train_data=X_train_with_featured, columns=continuous_columns)
X_train_with_featured = scale_data(X_train_with_featured, continuous_columns, scaler)
print(X_train_with_featured.head())
X_train_with_featured.shape

       credit_amount  gender  education  marital_status  age  sept_delay  \
3355      616.442211       2          2               1   48           0   
28151     547.723470       2          1               2   32           0   
8934      574.457135       1          2               1   34          -1   
13540     316.229347       1          2               1   28          -1   
14378     223.609034       1          3               2   26           2   

       august_delay  july_delay  june_delay  may_delay  ...  july_bill  \
3355              0           0           0          0  ...     363343   
28151             0           0           0          0  ...     199223   
8934             -1          -2          -2         -2  ...          0   
13540            -1          -2          -2         -2  ...          0   
14378             0           0           2          2  ...      45930   

       june_bill  may_bill  april_bill  sept_payment  august_payment  \
3355      290371    289600

(20975, 23)

In [7]:
from feature_preparation.feature_selection import *
X_train_with_featured = drop_correlated_features(X_train_with_featured)

In [8]:
X_train_with_featured.shape

(20975, 17)

In [9]:
from model_selection.model_training import *
train_decision_tree(X_train,y_train,"dec_tree_with_feature_engineering")
train_knn(X_train,y_train,"knn_with_feature_engineering")
train_svm(X_train,y_train,"svm_with_feature_engineering")
train_xgb(X_train,y_train,"xgb_feature_engineering")
train_logistic_regression(X_train,y_train,"log_reg_with_feature_engineering")


dec_tree_with_feature_engineering complete and saved
knn_with_feature_engineering complete and saved
svm_with_feature_engineering complete and saved
xgb_feature_engineering complete and saved
log_reg_with_feature_engineering complete and saved


c:\Users\banji\creditCard_default_risk\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---

In [ ]:
tree = joblib.load("../models/dec_tree_with_feature_engineering.joblib")
knn = joblib.load("../models/knn_with_feature_engineering.joblib")
svm = joblib.load("../models/svm_with_feature_engineering.joblib")
xgb = joblib.load("../models/xgb_feature_engineering.joblib")
log_reg = joblib.load("../models/log_reg_with_feature_engineering.joblib")


,penalty,'l2'
,dual,False
,tol,0.0001
,C,30
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,70
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'
